In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# importing libraries:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from lightgbm import LGBMClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import warnings 
warnings.filterwarnings('ignore')

# Loading Data:

In [ ]:
train_data = pd.read_csv(r"/kaggle/input/spaceship-titanic/train.csv")
test_data = pd.read_csv(r"/kaggle/input/spaceship-titanic/test.csv")
sample_submission_data = pd.read_csv(r"/kaggle/input/spaceship-titanic/sample_submission.csv")

# checking shape of the data:
print("train_data shape :",train_data.shape)
print('test_data shape :',test_data.shape)
print("sample_submission_data shape :",sample_submission_data.shape)
train_data.head()

In [ ]:
train_data[['DeckName','DeckNumber','DeckSide']]=train_data['Cabin'].str.split('/',expand=True)
test_data[['DeckName','DeckNumber','DeckSide']]=test_data['Cabin'].str.split('/',expand=True)
train_data = train_data.drop(['Cabin'],axis=1)
test_data = test_data.drop(['Cabin'],axis=1)

train_data[['FName', 'LName']] = train_data.Name.str.split(" ", expand=True) 
test_data[['FName', 'LName']] = test_data.Name.str.split(" ", expand=True) 

Total_spent = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']

train_data['Spent_Amount'] = train_data.loc[:,Total_spent].sum(axis=1)
test_data['Spent_Amount'] = test_data.loc[:,Total_spent].sum(axis=1)

print(train_data.shape)
print(test_data.shape)

In [ ]:
train_data.head(2)

In [ ]:
test_data.head(2)

In [ ]:
# train data information:
#train_data.info()

In [ ]:
# checking null values:
train_data.isnull().sum().sort_values(ascending=False)

In [ ]:
train_data.drop_duplicates(inplace=True)
train_data.shape

In [ ]:
# Calculate the percentage of missing values in each feature:
missing_percentage = train_data.isnull().mean() * 100

# Create a bar plot
plt.figure(figsize=(15, 6))
missing_percentage.plot(kind='bar', color='skyblue')
plt.title('Percentage of Missing Values in Features')
plt.xlabel('Features')
plt.ylabel('Percentage (%)')
plt.show()

In [ ]:
train_data = train_data.dropna()
train_data.isnull().sum().sum()

In [ ]:
# Description of the train data:
train_data.describe()

In [ ]:
train_data.hist(figsize=(10,10),color = 'blue')
plt.show()

In [ ]:
for col_name in train_data.columns:
    if train_data[col_name].dtypes=='object':
        train_data[col_name] = train_data[col_name].fillna(train_data[col_name].mode()[0])
    else:
        train_data[col_name] = train_data[col_name].fillna(train_data[col_name].median())
print(train_data.shape)

In [ ]:
for col_name in test_data.columns:
    if test_data[col_name].dtypes=='object':
        test_data[col_name] = test_data[col_name].fillna(test_data[col_name].mode()[0])
    else:
        test_data[col_name] = test_data[col_name].fillna(test_data[col_name].median())
print(test_data.shape)

train_data.dropna(inplace=True)
train_data.shape

In [ ]:
# Create a TF-IDF vectorizer.
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
# Fit and transform the training data.
X_train_LName = tfidf_vectorizer.fit_transform(train_data['LName'])
# Transform the testing data.
X_test_LName = tfidf_vectorizer.transform(test_data['LName'])

print(X_train_LName.shape)
print(X_test_LName.shape)

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(5)
xtrain = svd.fit_transform(X_train_LName)
xtest = svd.transform(X_test_LName)

# Convert to DataFrames
tfidf_train = pd.DataFrame(xtrain)
tfidf_test = pd.DataFrame(xtest)

# Naming columns in the new DataFrames
cols = [('LName' + "_tfidf_" + str(f)) for f in tfidf_train.columns.to_list()]
tfidf_train.columns = cols
tfidf_test.columns = cols
# Reset the index of the DataFrames before concatenation
df_train = train_data.reset_index(drop=True)
df_test = test_data.reset_index(drop=True)
# Concatenate transformed features with original data
train_data = pd.concat([train_data, tfidf_train], axis="columns")
test_data = pd.concat([test_data, tfidf_test], axis="columns")

In [ ]:
#train_data = train_data.drop(['LName','FName','Name'],axis=1,inplace=True)
#test_data = test_data.drop(['LName','FName','Name'],axis=1,inplace=True)
train_data.head()

In [ ]:
train_data['Transported'].value_counts()

In [ ]:
train_data.info()

# Boxplot:

In [ ]:
train_data.boxplot(figsize=(20,5))
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
#  object datatype columns encoding:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
object_columns = train_data.select_dtypes(include='object').columns.difference(['PassengerId'])
for col_name in object_columns:
    if train_data[col_name].dtypes=='object':
        train_data[col_name]=labelencoder.fit_transform(train_data[col_name])

In [ ]:
object_columns = test_data.select_dtypes(include='object').columns.difference(['PassengerId'])
labelencoder = LabelEncoder()
for column in object_columns:
    test_data[column] = labelencoder.fit_transform(test_data[column])

# Heatmap:

In [ ]:
plt.figure(figsize=(18,12))
sns.heatmap(train_data.corr(), annot=True,cmap="coolwarm")

# Data splitting and Model Building:

In [ ]:
train_data.head(2)

In [ ]:
test_data.head(2)

In [ ]:
#train_data.corr()['Transported'].drop('Transported').sort_values(ascending=False)

In [ ]:
param = {'colsample_bytree': 0.8, 
                  'learning_rate': 0.0218798, 
                  'min_child_samples': 10, 
                  'n_estimators': 200, 
                  'num_leaves': 31, 
                  'subsample': 0.8}
lgb_params = {'colsample_bytree': 0.7774799983649324, 
              'learning_rate': 0.007653648135411494, 
              'max_depth': 5, 
              'n_estimators': 350, 
              'reg_alpha': 0.14326300616140863, 
              'reg_lambda': 0.9310129332502252, 
              'subsample': 0.6189257947519665}

In [ ]:
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

X = train_data.drop(['Transported','PassengerId','Name','VIP','FName'], axis=1)
y = train_data['Transported']


# Initialize StratifiedKFold
num_folds = 10
stratified_kfold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

lgb_model = LGBMClassifier(**lgb_params,verbose=100)
cat_model = CatBoostClassifier(learning_rate = 0.05,iterations = 100,verbose  = 300)

lgb_acc = []
cat_acc = []

# Iterate over the folds
for fold, (train_idx, valid_idx) in enumerate(stratified_kfold.split(X, y)):
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

    # Train and predict using LGBMClassifier
    lgb_model.fit(X_train, y_train)
    lgb_preds = lgb_model.predict(X_valid)
    lgb_accuracy = accuracy_score(y_valid, lgb_preds)
    lgb_acc.append(lgb_accuracy)

    # Train and predict using CatBoostClassifier
    cat_model.fit(X_train, y_train)
    catboost_preds = cat_model.predict(X_valid)
    # Convert y_pred from string to boolean
    y_pred = np.array([True if pred == 'True' else False for pred in catboost_preds])
    cat_accuracy = accuracy_score(y_valid, y_pred)
    cat_acc.append(cat_accuracy)

    print(f"Fold {fold+1} - LGB Accuracy: {lgb_accuracy:.4f}, CatBoost Accuracy: {cat_accuracy:.4f}")

# Calculate mean accuracy across folds
mean_lgb_accuracy = sum(lgb_acc) / num_folds
mean_catboost_accuracy = sum(cat_acc) / num_folds
f1_lgb = f1_score(y_valid, lgb_preds, average='weighted')
f1_cat = f1_score(y_valid, y_pred, average='weighted')

print(f"\nMean LGB Accuracy: {mean_lgb_accuracy:.4f}")
print(f"Mean CatBoost Accuracy: {mean_catboost_accuracy:.4f}")

print(f"\nMean LGB F1: {f1_lgb:.4f}")
print(f"Mean CatBoost F1: {f1_cat:.4f}")

In [ ]:
X_test = test_data.drop(['PassengerId','Name','VIP','FName'], axis=1)
lgb_pred = lgb_model.predict(X_test)
lgb_pred = lgb_pred.astype(bool)
submission1 = pd.DataFrame({'PassengerId': test_data.PassengerId,'Transported': lgb_pred})
submission1.to_csv('submission1.csv', index=False)
submission1['Transported'].value_counts()

In [ ]:
X_test = test_data.drop(['PassengerId','Name','VIP','FName'], axis=1)
cat_pred = cat_model.predict(X_test)
cat_pred = cat_pred.astype(bool)
submission2 = pd.DataFrame({'PassengerId': test_data.PassengerId,'Transported': cat_pred.flatten()})
submission2.to_csv('submission2.csv', index=False)
submission2['Transported'].value_counts()

In [ ]:
submission2.head()

In [ ]:
X_train = train_data.drop(['Transported','PassengerId','Name','FName','VIP'], axis=1)
y_train = train_data['Transported']
X_test = test_data.drop(['PassengerId','Name','FName','VIP'], axis=1)

# Data Scaling:

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# LGBMClassifier:

In [ ]:
param = {'colsample_bytree': 0.8, 
                  'learning_rate': 0.02102878, 
                  'min_child_samples': 10, 
                  'n_estimators': 200, 
                  'num_leaves': 31, 
                  'subsample': 0.8}
param1 = {'colsample_bytree': 0.8, 
                  'learning_rate': 0.0197789, 
                  'min_child_samples': 10, 
                  'n_estimators': 200, 
                  'num_leaves': 31, 
                  'subsample': 0.8}
model_lgb1 = lgb.LGBMClassifier(**param)
model_lgb1.fit(X_train, y_train)
lgb_pred1=model_lgb1.predict(X_test)
lgb_pred1 = lgb_pred1.astype(bool)
# Get value counts of predictions
print(pd.Series(lgb_pred1).value_counts())

submission = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Transported': lgb_pred1})
print(submission.head())
submission.to_csv('submission.csv', index=False)

param = {'colsample_bytree': 0.8, 
              'learning_rate': 0.02, 
              'min_child_samples': 10, 
              'n_estimators': 200, 
              'num_leaves': 31, 
              'subsample': 0.8}

param1 = {'colsample_bytree': 0.8, 
                  'learning_rate': 0.0218798, 
                  'min_child_samples': 10, 
                  'n_estimators': 200, 
                  'num_leaves': 31, 
                  'subsample': 0.8}

Hyperparameters = {'num_leaves': 300, 
                   'learning_rate': 0.2, 
                   'feature_fraction': 0.8, 
                   'bagging_fraction': 0.99, 
                   'max_depth': 12, 
                   'lambda_l1': 9.8, 
                   'lambda_l2': 0.75}
model_lgb = LGBMClassifier(**param1)
model_lgb.fit(X_train, y_train)
lgb_pred=model_lgb.predict(X_test)
# Get value counts of predictions
print(pd.Series(lgb_pred).value_counts())

submission1 = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Transported': lgb_pred})
print(submission1.head())
submission1.to_csv('submission1.csv', index=False)

In [ ]:
sample_submission_data['Transported'].value_counts()